In [38]:
import pandas as pd
import mysql.connector
import os
import sqlalchemy
import datetime

# Connection Parameters

In [18]:
DB_HOSTNAME = 'localhost'
DB_NAME = 'TestDB'

DB_PW = os.environ.get('DB_PW')
DB_USERNAME = os.environ.get('DB_USER') # Make sure you have the environmental variables set before using

if (DB_PW is None) or (DB_USERNAME is None):
    raise Exception('DB_PW or DB_USERNAME are not defined as environmental variables')
else:
    print('Variables are valid')

Variables are valid


In [19]:
# Use module: "sqlalchemy"
def create_con_sqlalchemy():
    database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}{4}'.
                                                   format(DB_USERNAME, DB_PW, 
                                                          DB_HOSTNAME, DB_NAME,  "?charset=utf8mb4"))
    return database_connection

## Use module: "mysql.connector"
# def create_con_mysql():
#     con = mysql.connector.connect(
#         host=DB_HOSTNAME,
#         user=DB_USERNAME,
#         password=DB_PW,
#         database=DB_NAME)
#     return con


### Create Sample DF

In [20]:
df = pd.DataFrame([[1, 'Eilon']], columns=['RowID', 'Name'])
data = [[2, 'Shir'], [3, 'Ofir']]
df2 = pd.DataFrame(data, columns=list(df.columns))
df = df.append(df2, ignore_index=True)
df

,RowID,Name
0,1,Eilon
1,2,Shir
2,3,Ofir


### Read Hebrew CSV

In [21]:
path_csv = r"C:\Users\eilon.eilstein\Desktop\Eilon\Python\TestData.csv"
heb_df = pd.read_csv(path_csv, encoding='ISO-8859-8', parse_dates=['תאריך לידה'])
heb_df.loc[len(heb_df)] = ['אופיר', 'ברק', '0', pd.to_datetime(datetime.datetime.today().date())]
heb_df

,שם,שם משפחה,גיל,תאריך לידה
0,אילון,איילשטיין,22,1999-07-20
1,שיר,ברק,22,1999-06-13
2,אופיר,ברק,0,2022-05-02


# sql_to_python

In [34]:
def sql_to_python(table_name):
    con = create_con_sqlalchemy()
    qry = f'SELECT * from {table_name}'
    df = pd.read_sql_query(qry, con=con)
    print(f"The SQL table '{table_name}' is now saved as Pandas DataFrame")
    return df

# python_to_sql

In [35]:
def python_to_sql(df, table_name='test'):
    df.to_sql(con=create_con_sqlalchemy(),
              name=table_name,
              if_exists='replace', 
              index=False) 
    print(f"Pandas DataFrame is now saved as '{table_name}' in SQL")

In [36]:
python_to_sql(heb_df, 'TestTable2')

Pandas DataFrame is now saved as 'TestTable2' in SQL


In [37]:
new_df = sql_to_python('TestTable')
new_df.head()

The SQL table 'TestTable' is now saved as Pandas DataFrame


,RowID,Name
0,1,Eilon
1,2,Shir
2,3,Ofir
